<a href="https://colab.research.google.com/github/james330/NLPHW05/blob/main/NLPHW05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencc-python-reimplemented

In [20]:
import json
from opencc import OpenCC
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam # - Works

import matplotlib.pyplot as plt  # for 畫圖用

In [3]:
# 將json格式，轉為python格式
# {'english':'xxxx' ,'chinese':'xxxxx'}
data = []
datas = []
file = open('./drive/MyDrive/Colab Notebooks/data/translation2019zh_train.json' ,'r')
for f in file:
  data.append(json.loads(f)) #就這樣將json 轉成python了

In [4]:
# 分割英文數據和中文數據
en_data = []
ch_data = []
for da in data:
  en_data.append(da['english'])
  ch_data.append('\t'+da['chinese']+'\n')

print('英文數據:\n', en_data[:10])
print('\n中文數據:\n', ch_data[:3])

英文數據:
 ['For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.', 'He calls the Green Book, his book of teachings, “the new gospel.', 'And the light breeze moves me to caress her long ear', 'They have the blood of martyrs is the White to flow …', "Finally, the Lakers head to the Motor City to take on a Pistons team that currently owns the Eastern Conference's second best record (1/31). L.", '"The perfect match—my father loves names and Jackie loves money, " sneered Alexander at the wedding. Neither he nor Christina ever got along with their stepmother17.', 'In 2006, Walmart was charged with racism when its recommendation engine paired Planet of the Apes with a documentary about Martin Luther King.', 'The matte as main copper phase in the cleaning. slag was deter- mined by electron probe microscopic analysis.', 'Have you shined your shoes?', 'The Tanning Matrix can be formed by resorcinol and oxazolidine E, and the reactioncharater

In [5]:
# 分別生成中英文字典，英文
i = 0
en_vocab = set()
for ed in en_data:
  if i == 200:
    break
  else:
    en_vocab = en_vocab|set(''.join(ed))
    i+=1

id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}
print('\n英文字典:\n', en2id)


英文字典:
 {'—': 0, 'L': 1, '’': 2, 'N': 3, 'U': 4, '–': 5, 'R': 6, 'k': 7, '2': 8, 'T': 9, 'q': 10, 'z': 11, ')': 12, 'C': 13, 'J': 14, 'p': 15, '7': 16, 'l': 17, 'n': 18, 'c': 19, 'O': 20, 'G': 21, '4': 22, 'o': 23, '”': 24, ':': 25, '[': 26, 'A': 27, '6': 28, 'b': 29, 'g': 30, 'd': 31, 'Y': 32, 'u': 33, '?': 34, 'a': 35, "'": 36, '!': 37, 'M': 38, ']': 39, 'e': 40, '.': 41, '"': 42, '_': 43, 'E': 44, '$': 45, ' ': 46, 'D': 47, '1': 48, 'W': 49, '…': 50, '-': 51, '~': 52, 'I': 53, 'Q': 54, '“': 55, 'x': 56, 'j': 57, 'h': 58, '9': 59, 'r': 60, 'S': 61, '<': 62, 'Z': 63, '(': 64, 'X': 65, 's': 66, 'i': 67, '·': 68, '3': 69, 'w': 70, '8': 71, 'K': 72, ',': 73, 'f': 74, 'P': 75, '%': 76, 'B': 77, 't': 78, ';': 79, '0': 80, 'F': 81, '/': 82, '&': 83, 'v': 84, 'V': 85, '5': 86, 'y': 87, 'm': 88, 'H': 89}


In [6]:
# 分別生成中英文字典，中文
i = 0
ch_vocab = set()
for cd in ch_data:
  if i == 10000:
    break
  else:
    ch_vocab = ch_vocab|set(''.join(cd))
    i+=1

id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}
print('\n中文字典共計\n:', ch2id)


中文字典共計
: {'踝': 0, '振': 1, '薛': 2, '梯': 3, '筝': 4, '毫': 5, '瘩': 6, '镍': 7, '报': 8, '鹬': 9, '杰': 10, '冗': 11, '挫': 12, '萦': 13, '薰': 14, '骡': 15, '衔': 16, '夹': 17, '勺': 18, 'l': 19, '用': 20, '魇': 21, '碾': 22, '渔': 23, '肆': 24, '琼': 25, '赋': 26, '总': 27, '丑': 28, '抱': 29, '扔': 30, '虫': 31, '鳃': 32, '使': 33, '光': 34, '敞': 35, '著': 36, '侃': 37, '郴': 38, '渗': 39, '沙': 40, '幔': 41, '真': 42, '屠': 43, '鼓': 44, '敬': 45, '臾': 46, '钽': 47, '驱': 48, '钧': 49, '鞣': 50, '挣': 51, '条': 52, '劫': 53, '惯': 54, '刹': 55, '擞': 56, '抵': 57, '铀': 58, 'D': 59, '担': 60, '献': 61, '茅': 62, '夯': 63, 'W': 64, '部': 65, '虑': 66, '屎': 67, '罐': 68, '针': 69, 'S': 70, '薯': 71, '玲': 72, '龄': 73, '姻': 74, '犁': 75, '啶': 76, '攥': 77, '糯': 78, '惹': 79, '巫': 80, '厚': 81, '盼': 82, '．': 83, '驹': 84, '踏': 85, '闹': 86, '悚': 87, '颚': 88, '铄': 89, '穿': 90, '囱': 91, '拭': 92, '槲': 93, '憎': 94, '媲': 95, '兑': 96, '永': 97, '促': 98, '晋': 99, '蕾': 100, '嵊': 101, '善': 102, '坠': 103, '釉': 104, '精': 105, '迪': 106, '靡': 107, '此': 108, '九': 109,

In [7]:
i = 0
en_num_data = []
e = []
for line in en_data:
  if i == 10:
    break
  else:
    for en in line:
      e.append(en2id[en])
    en_num_data.append(e)
    e = []
    i+=1

In [8]:
i = 0
ch_num_data = []
c = []
for line in ch_data:
  if i == 10:
    break
  else:
    for ch in line:
      c.append(ch2id[ch])
    ch_num_data.append(c)
    c = []
    i+=1

In [9]:
# ???
i = 0
de_num_data = []
for line in ch_data:
  if i == 10:
    break
  else:
    for ch in line[1:]:
      de_num_data.append(ch2id[ch])
    i+=1

In [10]:
# 獲取輸入輸出端的最大長度
max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[0])

max encoder length: 194
max decoder length: 65
index data:
 [89, 40, 46, 19, 35, 17, 17, 66, 46, 78, 58, 40, 46, 21, 60, 40, 40, 18, 46, 77, 23, 23, 7, 73, 46, 58, 67, 66, 46, 29, 23, 23, 7, 46, 23, 74, 46, 78, 40, 35, 19, 58, 67, 18, 30, 66, 73, 46, 55, 78, 58, 40, 46, 18, 40, 70, 46, 30, 23, 66, 15, 40, 17, 41]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
# =======預定義模型參數========
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.003
BATCH_SIZE = 100
EPOCHS = 200